In [1]:
from pandas import read_csv

In [2]:
# Criando um objeto Pandas
uberFile = read_csv("data/uber.csv")

In [3]:
type(uberFile)

pandas.core.frame.DataFrame

In [4]:
# Visualizando as primeiras linhas
uberFile.head(10)

,dispatching_base_number,date,active_vehicles,trips
0,B02512,1/1/2015,190,1132
1,B02765,1/1/2015,225,1765
2,B02764,1/1/2015,3427,29421
3,B02682,1/1/2015,945,7679
4,B02617,1/1/2015,1228,9537
5,B02598,1/1/2015,870,6903
6,B02598,1/2/2015,785,4768
7,B02617,1/2/2015,1137,7065
8,B02512,1/2/2015,175,875
9,B02682,1/2/2015,890,5506


In [5]:
# Tranformando o dataframe (Pandas) em um Dataframe (Spark)
uberDF = sqlContext.createDataFrame(uberFile)

In [6]:
type(uberDF)

pyspark.sql.dataframe.DataFrame

In [7]:
# Criando o RDD a partir do arquivo csv
uberRDD = sc.textFile("data/uber.csv")

In [8]:
type(uberRDD)

pyspark.rdd.RDD

In [9]:
# Total de registros
uberRDD.count()

355

In [10]:
# Primeiro registro
uberRDD.first()

'dispatching_base_number,date,active_vehicles,trips'

In [55]:
# 1- Quantos são e quais são as bases de carros do Uber (onde os carros ficam esperando passageiros)?

# Destacando a coluna 'dispatching_base_number':

bases = []
for col in uberDF.collect():
    bases.append(col['dispatching_base_number'])
    
basesRDD = sc.parallelize(bases)
type(basesRDD)

pyspark.rdd.RDD

In [56]:
# Realizando a contagem e printando:

qntd_bases = basesRDD.map(lambda palavra: (palavra, 1)).reduceByKey(lambda a, b: a+b)
qntd_bases.collect()

[('B02598', 59),
 ('B02617', 59),
 ('B02764', 59),
 ('B02682', 59),
 ('B02512', 59),
 ('B02765', 59)]

In [69]:
# 2- Qual o total de veículos que passaram pela base B02617?

linha_bases_veiculos = uberDF.select('dispatching_base_number', 'active_vehicles')
linhacomB02617 = linha_bases_veiculos.filter(linha_bases_veiculos['dispatching_base_number'] == 'B02617')

linhacomB02617.groupBy(linhacomB02617['dispatching_base_number']).agg({'active_vehicles':'sum'}).show()

+-----------------------+--------------------+
|dispatching_base_number|sum(active_vehicles)|
+-----------------------+--------------------+
|                 B02617|               79758|
+-----------------------+--------------------+



In [81]:
# 3- Qual o total de corridas por base? Apresente de forma decrescente.

linha_bases_veiculos.groupBy(
    linha_bases_veiculos['dispatching_base_number']).agg(
        {'active_vehicles':'sum'}).sort('dispatching_base_number', ascending=False).show()


+-----------------------+--------------------+
|dispatching_base_number|sum(active_vehicles)|
+-----------------------+--------------------+
|                 B02765|               22575|
|                 B02764|              217290|
|                 B02682|               71431|
|                 B02617|               79758|
|                 B02598|               58653|
|                 B02512|               13125|
+-----------------------+--------------------+

